# Tarea 4: Solución de Reactor PFR usando Elementos Finitos

## Problema: Estado Estacionario de Concentración en Reactor PFR

Este notebook resuelve la ecuación diferencial que modela el estado estacionario de la concentración de una sustancia con reacción de primer orden en un reactor PFR (Plug Flow Reactor) usando:

1. **Formulación NO discretizada**: Polinomios de Legendre
2. **Formulación discretizada (FEM)**: Polinomios de Lagrange

### Ecuación Gobernante

$$D\frac{d^2c}{dx^2} - U\frac{dc}{dx} - kc = 0$$

### Condiciones de Contorno

- En $x = 0$: $U \cdot c_{inlet} = U \cdot c - D\frac{dc}{dx}$
- En $x = L$: $\frac{dc}{dx} = 0$

### Parámetros del Problema

- $D = 5000$ m²/hr (coeficiente de difusión/dispersión)
- $U = 100$ m/hr (velocidad advectiva)
- $k = 2$ 1/hr (tasa de reacción)
- $L = 100$ m (longitud del reactor)
- $c_{inlet} = 100$ mol/L (concentración de entrada)

In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from numpy.polynomial.legendre import Legendre, leggauss
from scipy import sparse
from scipy.sparse.linalg import spsolve

# Configuración para gráficas
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Parámetros del Problema

In [ ]:
# Parámetros del reactor PFR
D = 5000.0      # [m²/hr] coeficiente de difusión/dispersión
U = 100.0       # [m/hr] velocidad advectiva
k = 2.0         # [1/hr] tasa de reacción
L = 100.0       # [m] longitud del reactor
c_inlet = 100.0 # [mol/L] concentración de entrada

print(f"Parámetros del Reactor PFR:")
print(f"  D = {D} m²/hr")
print(f"  U = {U} m/hr")
print(f"  k = {k} 1/hr")
print(f"  L = {L} m")
print(f"  c_inlet = {c_inlet} mol/L")

## 2. Solución Analítica

La solución analítica está dada por:

$$c(x) = \frac{U \cdot c_{in}}{(U - D\lambda_1) \lambda_2 e^{\lambda_2 L} - (U - D\lambda_2) \lambda_1 e^{\lambda_1 L}} \left[\lambda_2 e^{\lambda_2 L} e^{\lambda_1 x} - \lambda_1 e^{\lambda_1 L} e^{\lambda_2 x}\right]$$

donde:

$$\lambda_{1,2} = \frac{U}{2D}\left[1 \pm \sqrt{1 + \frac{4kD}{U^2}}\right]$$

In [ ]:
def solucion_analitica(x, D, U, k, L, c_inlet):
    """
    Calcula la solución analítica del reactor PFR.
    
    Parámetros:
    -----------
    x : array_like
        Posiciones donde evaluar la solución
    D : float
        Coeficiente de difusión [m²/hr]
    U : float
        Velocidad advectiva [m/hr]
    k : float
        Tasa de reacción [1/hr]
    L : float
        Longitud del reactor [m]
    c_inlet : float
        Concentración de entrada [mol/L]
    
    Retorna:
    --------
    c : array_like
        Concentración en cada posición x
    """
    # Cálculo de lambda1 y lambda2
    raiz = np.sqrt(1 + 4*k*D/U**2)
    lambda1 = (U/(2*D)) * (1 + raiz)
    lambda2 = (U/(2*D)) * (1 - raiz)
    
    # Términos del denominador
    denom = (U - D*lambda1) * lambda2 * np.exp(lambda2*L) - (U - D*lambda2) * lambda1 * np.exp(lambda1*L)
    
    # Solución
    c = (U * c_inlet / denom) * (lambda2 * np.exp(lambda2*L) * np.exp(lambda1*x) - 
                                   lambda1 * np.exp(lambda1*L) * np.exp(lambda2*x))
    
    return c

# Graficar solución analítica
x_plot = np.linspace(0, L, 500)
c_analitica = solucion_analitica(x_plot, D, U, k, L, c_inlet)

plt.figure(figsize=(10, 6))
plt.plot(x_plot, c_analitica, 'k-', lw=2, label='Solución Analítica')
plt.xlabel('Posición x [m]')
plt.ylabel('Concentración c [mol/L]')
plt.title('Solución Analítica del Reactor PFR')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print(f"Concentración en x=0: {c_analitica[0]:.4f} mol/L")
print(f"Concentración en x=L: {c_analitica[-1]:.4f} mol/L")

## 3. Desarrollo Matemático: Método de Galerkin con Debilitamiento

### 3.1 Formulación Débil

Partiendo de la ecuación diferencial:
$$D\frac{d^2c}{dx^2} - U\frac{dc}{dx} - kc = 0$$

Multiplicamos por una función de prueba $N_l(x)$ e integramos:
$$\int_0^L N_l \left(D\frac{d^2c}{dx^2} - U\frac{dc}{dx} - kc\right) dx = 0$$

### 3.2 Integración por Partes (Debilitamiento)

Aplicando integración por partes al término de segunda derivada:
$$\int_0^L N_l D\frac{d^2c}{dx^2} dx = \left[N_l D\frac{dc}{dx}\right]_0^L - \int_0^L \frac{dN_l}{dx} D\frac{dc}{dx} dx$$

### 3.3 Aproximación de Galerkin

Aproximamos la solución como:
$$c(x) \approx \sum_{m=0}^{M-1} a_m N_m(x)$$

### 3.4 Sistema Matricial $[K]\{a\} = \{F\}$

Donde:
$$K_{lm} = -\int_0^L D\frac{dN_l}{dx}\frac{dN_m}{dx} dx - \int_0^L U N_l \frac{dN_m}{dx} dx - \int_0^L k N_l N_m dx$$

$$F_l = -\left[N_l D\frac{dc}{dx}\right]_0^L$$

### 3.5 Condiciones de Contorno

- En $x=0$: $D\frac{dc}{dx} = U(c - c_{inlet})$
- En $x=L$: $\frac{dc}{dx} = 0$

Por lo tanto:
$$F_l = N_l(0) \cdot U \cdot c_{inlet}$$

## 4. Funciones Auxiliares de Integración

In [ ]:
def gauss_legendre_integral(f, a, b, n=80):
    """Integra f(x) en [a,b] usando cuadratura Gauss-Legendre con n puntos."""
    xi, wi = leggauss(n)  # puntos y pesos en [-1,1]
    # Mapeo lineal a [a,b]
    x_mapped = 0.5 * (b - a) * xi + 0.5 * (b + a)
    w_mapped = 0.5 * (b - a) * wi
    return np.dot(w_mapped, f(x_mapped))

## 5. Formulación NO Discretizada: Polinomios de Legendre

### 5.1 Funciones Base: Polinomios de Legendre

Usamos polinomios de Legendre $P_m(\xi)$ donde $\xi \in [-1, 1]$ mapeados al dominio físico $[0, L]$:

$$x = \frac{L}{2}(\xi + 1), \quad \xi = \frac{2x}{L} - 1$$

In [ ]:
# ============================================================================
# FORMULACIÓN NO DISCRETIZADA CON LEGENDRE
# ============================================================================

def Nm_legendre(x, m, L):
    """Polinomio de Legendre de orden m evaluado en x (dominio físico [0,L])."""
    xi = 2*x/L - 1  # Mapeo de [0,L] a [-1,1]
    return Legendre.basis(m)(xi)

def dNm_legendre(x, m, L):
    """Primera derivada del polinomio de Legendre respecto a x."""
    xi = 2*x/L - 1
    # d/dx = d/dxi * dxi/dx = d/dxi * (2/L)
    return Legendre.basis(m).deriv()(xi) * (2/L)

def Ke_legendre_nodiscr(l, m, D, U, k, L, quad_n=100):
    """
    Calcula el elemento K_lm de la matriz de rigidez (no discretizada).
    
    K_lm = -∫ D·dNl/dx·dNm/dx dx - ∫ U·Nl·dNm/dx dx - ∫ k·Nl·Nm dx
    """
    # Término difusivo: -D * ∫ dNl/dx * dNm/dx dx
    integrand1 = lambda x: dNm_legendre(x, l, L) * dNm_legendre(x, m, L)
    I1 = gauss_legendre_integral(integrand1, 0, L, n=quad_n)
    
    # Término advectivo: -U * ∫ Nl * dNm/dx dx
    integrand2 = lambda x: Nm_legendre(x, l, L) * dNm_legendre(x, m, L)
    I2 = gauss_legendre_integral(integrand2, 0, L, n=quad_n)
    
    # Término reactivo: -k * ∫ Nl * Nm dx
    integrand3 = lambda x: Nm_legendre(x, l, L) * Nm_legendre(x, m, L)
    I3 = gauss_legendre_integral(integrand3, 0, L, n=quad_n)
    
    return -D*I1 - U*I2 - k*I3

def Fe_legendre_nodiscr(l, U, c_inlet, L):
    """
    Calcula el elemento F_l del vector de carga (no discretizada).
    
    De las condiciones de contorno:
    F_l = U * c_inlet * Nl(0)
    """
    # Evaluar Nl en x=0
    Nl_at_0 = Nm_legendre(0.0, l, L)
    return U * c_inlet * Nl_at_0

def assemble_KF_legendre_nodiscr(M, D, U, k, L, c_inlet, quad_n=100):
    """Ensambla las matrices K y F globales (no discretizada)."""
    K = np.zeros((M, M))
    F = np.zeros(M)
    
    for l in range(M):
        for m in range(M):
            K[l, m] = Ke_legendre_nodiscr(l, m, D, U, k, L, quad_n=quad_n)
        F[l] = Fe_legendre_nodiscr(l, U, c_inlet, L)
    
    return K, F

def solve_galerkin_legendre(M, D, U, k, L, c_inlet, x_eval, quad_n=100):
    """
    Resuelve el problema usando Galerkin con polinomios de Legendre.
    
    Retorna:
    --------
    a : array
        Coeficientes de la aproximación
    c_approx : array
        Concentración aproximada en x_eval
    """
    # Ensamblar sistema
    K, F = assemble_KF_legendre_nodiscr(M, D, U, k, L, c_inlet, quad_n=quad_n)
    
    # Resolver sistema lineal
    a = np.linalg.solve(K, F)
    
    # Evaluar aproximación
    c_approx = np.zeros_like(x_eval)
    for m in range(M):
        c_approx += a[m] * Nm_legendre(x_eval, m, L)
    
    return a, c_approx

### 5.2 Solución con Diferentes Valores de M

In [ ]:
# Valores de M a probar
M_values = [3, 5, 8, 12, 15]

# Puntos de evaluación
x_plot = np.linspace(0, L, 400)
c_exact = solucion_analitica(x_plot, D, U, k, L, c_inlet)

# Graficar soluciones para diferentes M
plt.figure(figsize=(12, 7))

colors = plt.cm.viridis(np.linspace(0, 0.9, len(M_values)))

for idx, M in enumerate(M_values):
    a, c_approx = solve_galerkin_legendre(M, D, U, k, L, c_inlet, x_plot, quad_n=120)
    plt.plot(x_plot, c_approx, '-', color=colors[idx], lw=1.5, label=f'M = {M}')

plt.plot(x_plot, c_exact, 'k--', lw=2, label='Solución Analítica')
plt.xlabel('Posición x [m]', fontsize=12)
plt.ylabel('Concentración c [mol/L]', fontsize=12)
plt.title('Método de Galerkin con Polinomios de Legendre (No Discretizado)', fontsize=13)
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.3 Estudio de Convergencia (No Discretizado)

Analizamos cómo el error disminuye al aumentar M.

In [ ]:
# Estudio de convergencia
M_convergencia = np.arange(3, 25, 1)
errores_L2 = []
errores_max = []

x_error = np.linspace(0, L, 500)
c_exact_error = solucion_analitica(x_error, D, U, k, L, c_inlet)

for M in M_convergencia:
    try:
        a, c_approx = solve_galerkin_legendre(M, D, U, k, L, c_inlet, x_error, quad_n=120)
        
        # Error L2
        error_L2 = np.sqrt(np.trapz((c_approx - c_exact_error)**2, x_error) / L)
        errores_L2.append(error_L2)
        
        # Error máximo
        error_max = np.max(np.abs(c_approx - c_exact_error))
        errores_max.append(error_max)
    except:
        errores_L2.append(np.nan)
        errores_max.append(np.nan)

# Gráficas de convergencia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogy(M_convergencia, errores_L2, 'bo-', lw=2, markersize=6)
ax1.set_xlabel('Número de funciones base (M)', fontsize=12)
ax1.set_ylabel('Error L2', fontsize=12)
ax1.set_title('Convergencia: Error L2 vs M (No Discretizado)', fontsize=12)
ax1.grid(True, alpha=0.3)

ax2.semilogy(M_convergencia, errores_max, 'ro-', lw=2, markersize=6)
ax2.set_xlabel('Número de funciones base (M)', fontsize=12)
ax2.set_ylabel('Error Máximo', fontsize=12)
ax2.set_title('Convergencia: Error Máximo vs M (No Discretizado)', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nResultados de Convergencia (No Discretizado):")
print(f"  M = {M_convergencia[0]}: Error L2 = {errores_L2[0]:.6e}, Error Max = {errores_max[0]:.6e}")
print(f"  M = {M_convergencia[-1]}: Error L2 = {errores_L2[-1]:.6e}, Error Max = {errores_max[-1]:.6e}")
print(f"  Reducción de error: {errores_L2[0]/errores_L2[-1]:.2f}x")

### 5.4 Análisis del Residuo (No Discretizado)

El residuo se calcula como:
$$R(x) = D\frac{d^2c_{approx}}{dx^2} - U\frac{dc_{approx}}{dx} - k \cdot c_{approx}$$

In [ ]:
def d2Nm_legendre(x, m, L):
    """Segunda derivada del polinomio de Legendre respecto a x."""
    xi = 2*x/L - 1
    # d²/dx² = d²/dxi² * (dxi/dx)² = d²/dxi² * (2/L)²
    return Legendre.basis(m).deriv(2)(xi) * (2/L)**2

def calcular_residuo(a, x, D, U, k, L):
    """Calcula el residuo de la ecuación diferencial."""
    M = len(a)
    
    # Inicializar términos
    c = np.zeros_like(x)
    dc_dx = np.zeros_like(x)
    d2c_dx2 = np.zeros_like(x)
    
    # Calcular c, dc/dx y d²c/dx²
    for m in range(M):
        c += a[m] * Nm_legendre(x, m, L)
        dc_dx += a[m] * dNm_legendre(x, m, L)
        d2c_dx2 += a[m] * d2Nm_legendre(x, m, L)
    
    # Residuo: D*d²c/dx² - U*dc/dx - k*c
    R = D*d2c_dx2 - U*dc_dx - k*c
    
    return R

# Calcular residuo para diferentes M
M_residuo = [5, 8, 12, 15]
x_residuo = np.linspace(0, L, 400)

plt.figure(figsize=(12, 7))

colors_res = plt.cm.plasma(np.linspace(0, 0.9, len(M_residuo)))

for idx, M in enumerate(M_residuo):
    a, _ = solve_galerkin_legendre(M, D, U, k, L, c_inlet, x_residuo, quad_n=120)
    R = calcular_residuo(a, x_residuo, D, U, k, L)
    plt.plot(x_residuo, R, '-', color=colors_res[idx], lw=1.5, label=f'M = {M}')

plt.xlabel('Posición x [m]', fontsize=12)
plt.ylabel('Residuo R(x)', fontsize=12)
plt.title('Residuo de la Ecuación Diferencial (No Discretizado)', fontsize=13)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Formulación Discretizada: Elementos Finitos con Lagrange

### 6.1 Discretización del Dominio

Dividimos el dominio $[0, L]$ en $n_{elem}$ elementos:
- Cada elemento tiene longitud $h_e$
- Usamos **polinomios de Lagrange lineales** como funciones base
- Cada elemento tiene 2 nodos

### 6.2 Funciones Base de Lagrange (Lineales)

En el elemento local $[0, h_e]$:
$$N_1(\xi) = 1 - \xi, \quad N_2(\xi) = \xi, \quad \xi \in [0, 1]$$

In [ ]:
# ============================================================================
# FORMULACIÓN DISCRETIZADA CON LAGRANGE (FEM)
# ============================================================================

def create_mesh(L, n_elem):
    """Crea una malla uniforme de elementos finitos."""
    nodes = np.linspace(0, L, n_elem + 1)
    h = L / n_elem
    
    # Conectividad: cada elemento tiene 2 nodos
    connectivity = np.zeros((n_elem, 2), dtype=int)
    for e in range(n_elem):
        connectivity[e, :] = [e, e+1]
    
    return nodes, h, connectivity

def N_lagrange_local(xi, i):
    """Funciones de forma de Lagrange lineales en coordenada local ξ ∈ [0,1]."""
    if i == 0:
        return 1 - xi
    elif i == 1:
        return xi
    else:
        raise ValueError("i debe ser 0 o 1 para elementos lineales")

def dN_lagrange_local(xi, i):
    """Derivada de las funciones de forma respecto a ξ."""
    if i == 0:
        return -1.0
    elif i == 1:
        return 1.0
    else:
        raise ValueError("i debe ser 0 o 1 para elementos lineales")

def Ke_lagrange_local(h_e, D, U, k, quad_n=10):
    """
    Calcula la matriz de rigidez local para un elemento.
    
    K^e_ij = -∫ D·dNi/dx·dNj/dx dx - ∫ U·Ni·dNj/dx dx - ∫ k·Ni·Nj dx
    
    Retorna matriz 2x2
    """
    K_local = np.zeros((2, 2))
    
    # Cuadratura Gauss-Legendre en [0,1]
    xi_gauss, w_gauss = leggauss(quad_n)
    xi_gauss = 0.5 * (xi_gauss + 1)  # Mapear de [-1,1] a [0,1]
    w_gauss = 0.5 * w_gauss
    
    for i in range(2):
        for j in range(2):
            # Término difusivo: -D * ∫ dNi/dx * dNj/dx dx
            # dN/dx = dN/dξ * dξ/dx = dN/dξ * (1/h_e)
            term1 = 0.0
            for xi, w in zip(xi_gauss, w_gauss):
                dNi_dx = dN_lagrange_local(xi, i) / h_e
                dNj_dx = dN_lagrange_local(xi, j) / h_e
                term1 += w * dNi_dx * dNj_dx * h_e
            
            # Término advectivo: -U * ∫ Ni * dNj/dx dx
            term2 = 0.0
            for xi, w in zip(xi_gauss, w_gauss):
                Ni = N_lagrange_local(xi, i)
                dNj_dx = dN_lagrange_local(xi, j) / h_e
                term2 += w * Ni * dNj_dx * h_e
            
            # Término reactivo: -k * ∫ Ni * Nj dx
            term3 = 0.0
            for xi, w in zip(xi_gauss, w_gauss):
                Ni = N_lagrange_local(xi, i)
                Nj = N_lagrange_local(xi, j)
                term3 += w * Ni * Nj * h_e
            
            K_local[i, j] = -D*term1 - U*term2 - k*term3
    
    return K_local

def assemble_global_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=10):
    """
    Ensambla la matriz de rigidez global y el vector de carga.
    """
    n_nodes = len(nodes)
    n_elem = len(connectivity)
    h_e = nodes[1] - nodes[0]  # Asumiendo malla uniforme
    
    # Inicializar matrices globales
    K_global = np.zeros((n_nodes, n_nodes))
    F_global = np.zeros(n_nodes)
    
    # Ensamblar matriz de rigidez
    for e in range(n_elem):
        # Obtener nodos del elemento
        node_ids = connectivity[e, :]
        
        # Matriz local
        K_local = Ke_lagrange_local(h_e, D, U, k, quad_n=quad_n)
        
        # Ensamblar en matriz global
        for i_local in range(2):
            i_global = node_ids[i_local]
            for j_local in range(2):
                j_global = node_ids[j_local]
                K_global[i_global, j_global] += K_local[i_local, j_local]
    
    # Aplicar condiciones de contorno en x=0
    # En x=0: U*c_inlet = U*c - D*dc/dx
    # Contribución al nodo 0: F[0] += U*c_inlet
    F_global[0] = U * c_inlet
    
    # Condición en x=L: dc/dx = 0 (natural BC, ya incluida)
    
    return K_global, F_global

def solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=10):
    """
    Resuelve el sistema FEM.
    """
    K, F = assemble_global_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=quad_n)
    
    # Resolver sistema lineal
    c_nodes = np.linalg.solve(K, F)
    
    return c_nodes

### 6.2 Solución con Diferentes Mallas

In [ ]:
# Definir diferentes mallas (refinamiento progresivo con factor ~1.2-1.5)
n_elem_list = [10, 15, 25, 40, 60]  # Incremento aproximado de 1.5x

# Puntos para solución exacta
x_exact = np.linspace(0, L, 500)
c_exact = solucion_analitica(x_exact, D, U, k, L, c_inlet)

# Resolver para cada malla
plt.figure(figsize=(12, 7))

colors_fem = plt.cm.coolwarm(np.linspace(0.1, 0.9, len(n_elem_list)))

for idx, n_elem in enumerate(n_elem_list):
    nodes, h, connectivity = create_mesh(L, n_elem)
    c_fem = solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=15)
    
    plt.plot(nodes, c_fem, 'o-', color=colors_fem[idx], lw=1.5, 
             markersize=4, label=f'FEM: {n_elem} elem (h={h:.2f} m)')

plt.plot(x_exact, c_exact, 'k--', lw=2, label='Solución Analítica')
plt.xlabel('Posición x [m]', fontsize=12)
plt.ylabel('Concentración c [mol/L]', fontsize=12)
plt.title('Método de Elementos Finitos con Lagrange (Discretizado)', fontsize=13)
plt.legend(fontsize=9, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 6.3 Estudio de Convergencia (Discretizado)

In [ ]:
# Estudio de convergencia FEM
n_elem_convergencia = [5, 10, 15, 20, 25, 30, 40, 50, 60, 80, 100]
errores_L2_fem = []
errores_max_fem = []
h_values = []

for n_elem in n_elem_convergencia:
    nodes, h, connectivity = create_mesh(L, n_elem)
    c_fem = solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=15)
    
    # Interpolar para comparar con solución exacta
    c_fem_interp = np.interp(x_exact, nodes, c_fem)
    
    # Error L2
    error_L2 = np.sqrt(np.trapz((c_fem_interp - c_exact)**2, x_exact) / L)
    errores_L2_fem.append(error_L2)
    
    # Error máximo
    error_max = np.max(np.abs(c_fem_interp - c_exact))
    errores_max_fem.append(error_max)
    
    h_values.append(h)

# Gráficas de convergencia
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Error L2 vs número de elementos
axes[0, 0].loglog(n_elem_convergencia, errores_L2_fem, 'bs-', lw=2, markersize=7)
axes[0, 0].set_xlabel('Número de elementos', fontsize=11)
axes[0, 0].set_ylabel('Error L2', fontsize=11)
axes[0, 0].set_title('Error L2 vs Número de Elementos (FEM)', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)

# Error L2 vs tamaño de malla h
axes[0, 1].loglog(h_values, errores_L2_fem, 'gs-', lw=2, markersize=7)
# Agregar línea de referencia O(h²)
h_ref = np.array(h_values)
axes[0, 1].loglog(h_ref, 0.5*h_ref**2, 'k--', alpha=0.5, label='O(h²)')
axes[0, 1].set_xlabel('Tamaño de malla h [m]', fontsize=11)
axes[0, 1].set_ylabel('Error L2', fontsize=11)
axes[0, 1].set_title('Error L2 vs Tamaño de Malla (FEM)', fontsize=12)
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Error máximo vs número de elementos
axes[1, 0].loglog(n_elem_convergencia, errores_max_fem, 'rs-', lw=2, markersize=7)
axes[1, 0].set_xlabel('Número de elementos', fontsize=11)
axes[1, 0].set_ylabel('Error Máximo', fontsize=11)
axes[1, 0].set_title('Error Máximo vs Número de Elementos (FEM)', fontsize=12)
axes[1, 0].grid(True, alpha=0.3)

# Error máximo vs tamaño de malla
axes[1, 1].loglog(h_values, errores_max_fem, 'ms-', lw=2, markersize=7)
axes[1, 1].loglog(h_ref, 0.5*h_ref**2, 'k--', alpha=0.5, label='O(h²)')
axes[1, 1].set_xlabel('Tamaño de malla h [m]', fontsize=11)
axes[1, 1].set_ylabel('Error Máximo', fontsize=11)
axes[1, 1].set_title('Error Máximo vs Tamaño de Malla (FEM)', fontsize=12)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nResultados de Convergencia (FEM):")
print(f"  {n_elem_convergencia[0]} elementos: Error L2 = {errores_L2_fem[0]:.6e}, Error Max = {errores_max_fem[0]:.6e}")
print(f"  {n_elem_convergencia[-1]} elementos: Error L2 = {errores_L2_fem[-1]:.6e}, Error Max = {errores_max_fem[-1]:.6e}")
print(f"  Reducción de error: {errores_L2_fem[0]/errores_L2_fem[-1]:.2f}x")

## 7. Comparación: No Discretizado vs Discretizado

In [ ]:
# Comparación directa de ambos métodos
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Solución con ambos métodos
M_comp = 15
n_elem_comp = 50

# No discretizado (Legendre)
a_leg, c_legendre = solve_galerkin_legendre(M_comp, D, U, k, L, c_inlet, x_exact, quad_n=120)

# Discretizado (Lagrange FEM)
nodes_fem, h_fem, conn_fem = create_mesh(L, n_elem_comp)
c_lagrange = solve_FEM(nodes_fem, conn_fem, D, U, k, c_inlet, quad_n=15)
c_lagrange_interp = np.interp(x_exact, nodes_fem, c_lagrange)

# Gráfica 1: Comparación de soluciones
axes[0].plot(x_exact, c_exact, 'k-', lw=2.5, label='Analítica')
axes[0].plot(x_exact, c_legendre, 'b--', lw=2, label=f'Legendre (M={M_comp})')
axes[0].plot(nodes_fem, c_lagrange, 'ro-', lw=1.5, markersize=3, 
             label=f'Lagrange FEM ({n_elem_comp} elem)')
axes[0].set_xlabel('Posición x [m]', fontsize=12)
axes[0].set_ylabel('Concentración c [mol/L]', fontsize=12)
axes[0].set_title('Comparación: Legendre vs Lagrange FEM', fontsize=13)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Gráfica 2: Errores
error_legendre = np.abs(c_legendre - c_exact)
error_lagrange = np.abs(c_lagrange_interp - c_exact)

axes[1].semilogy(x_exact, error_legendre, 'b-', lw=2, label=f'Legendre (M={M_comp})')
axes[1].semilogy(x_exact, error_lagrange, 'r--', lw=2, label=f'Lagrange FEM ({n_elem_comp} elem)')
axes[1].set_xlabel('Posición x [m]', fontsize=12)
axes[1].set_ylabel('Error Absoluto', fontsize=12)
axes[1].set_title('Comparación de Errores', fontsize=13)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nComparación de Métodos:")
print(f"  Legendre (M={M_comp}): Error L2 = {np.sqrt(np.trapz(error_legendre**2, x_exact)/L):.6e}")
print(f"  Lagrange FEM ({n_elem_comp} elem): Error L2 = {np.sqrt(np.trapz(error_lagrange**2, x_exact)/L):.6e}")

## 8. Estudio Paramétrico: Efectos de k

Analizamos cómo cambia el comportamiento del reactor al variar la tasa de reacción k.

In [ ]:
# Valores de k a estudiar
k_values = [k/4, k/2, k, 2*k, 4*k]  # [0.5, 1, 2, 4, 8] 1/hr

fig, axes = plt.subplots(2, 2, figsize=(15, 11))

# Colores para diferentes k
colors_k = plt.cm.Spectral(np.linspace(0.1, 0.9, len(k_values)))

# ===== Gráfica 1: Perfiles de concentración =====
for idx, k_val in enumerate(k_values):
    c_k = solucion_analitica(x_exact, D, U, k_val, L, c_inlet)
    axes[0, 0].plot(x_exact, c_k, '-', color=colors_k[idx], lw=2, 
                    label=f'k = {k_val:.2f} 1/hr')

axes[0, 0].set_xlabel('Posición x [m]', fontsize=12)
axes[0, 0].set_ylabel('Concentración c [mol/L]', fontsize=12)
axes[0, 0].set_title('Efecto de k en el Perfil de Concentración', fontsize=13)
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# ===== Gráfica 2: Concentración normalizada =====
for idx, k_val in enumerate(k_values):
    c_k = solucion_analitica(x_exact, D, U, k_val, L, c_inlet)
    c_norm = c_k / c_inlet
    axes[0, 1].plot(x_exact/L, c_norm, '-', color=colors_k[idx], lw=2, 
                    label=f'k = {k_val:.2f} 1/hr')

axes[0, 1].set_xlabel('Posición adimensional x/L', fontsize=12)
axes[0, 1].set_ylabel('Concentración normalizada c/c_inlet', fontsize=12)
axes[0, 1].set_title('Perfiles Normalizados', fontsize=13)
axes[0, 1].legend(fontsize=9)
axes[0, 1].grid(True, alpha=0.3)

# ===== Gráfica 3: Concentración a la salida vs k =====
k_range = np.linspace(0.1, 8, 100)
c_salida = []
for k_val in k_range:
    c_k = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)
    c_salida.append(c_k[0])

axes[1, 0].plot(k_range, c_salida, 'b-', lw=2.5)
axes[1, 0].axvline(k, color='r', linestyle='--', alpha=0.7, label=f'k nominal = {k} 1/hr')
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    axes[1, 0].plot(k_val, c_out, 'ro', markersize=8)

axes[1, 0].set_xlabel('Tasa de reacción k [1/hr]', fontsize=12)
axes[1, 0].set_ylabel('Concentración a la salida c(L) [mol/L]', fontsize=12)
axes[1, 0].set_title('Concentración de Salida vs k', fontsize=13)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# ===== Gráfica 4: Conversión del reactor vs k =====
conversion = [(c_inlet - c_out)/c_inlet * 100 for c_out in c_salida]

axes[1, 1].plot(k_range, conversion, 'g-', lw=2.5)
axes[1, 1].axvline(k, color='r', linestyle='--', alpha=0.7, label=f'k nominal = {k} 1/hr')
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    conv = (c_inlet - c_out)/c_inlet * 100
    axes[1, 1].plot(k_val, conv, 'ro', markersize=8)

axes[1, 1].set_xlabel('Tasa de reacción k [1/hr]', fontsize=12)
axes[1, 1].set_ylabel('Conversión [%]', fontsize=12)
axes[1, 1].set_title('Conversión del Reactor vs k', fontsize=13)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEstudio Paramétrico de k:")
print(f"{'k [1/hr]':>10} {'c(L) [mol/L]':>15} {'Conversión [%]':>18}")
print("-" * 50)
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    conv = (c_inlet - c_out)/c_inlet * 100
    print(f"{k_val:10.2f} {c_out:15.4f} {conv:18.2f}")

### 8.2 Análisis de Convergencia para Diferentes k

In [ ]:
# Estudiar convergencia para diferentes valores de k
k_conv_values = [k/2, k, 2*k]
M_test = 15
n_elem_test = 40

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Legendre con diferentes k
for idx, k_val in enumerate(k_conv_values):
    M_range = np.arange(3, 20, 2)
    errors_M = []
    
    for M in M_range:
        a, c_approx = solve_galerkin_legendre(M, D, U, k_val, L, c_inlet, x_exact, quad_n=120)
        c_ex = solucion_analitica(x_exact, D, U, k_val, L, c_inlet)
        error = np.sqrt(np.trapz((c_approx - c_ex)**2, x_exact) / L)
        errors_M.append(error)
    
    axes[0].semilogy(M_range, errors_M, 'o-', lw=2, label=f'k = {k_val:.2f} 1/hr')

axes[0].set_xlabel('Número de funciones base M', fontsize=12)
axes[0].set_ylabel('Error L2', fontsize=12)
axes[0].set_title('Convergencia Legendre para diferentes k', fontsize=13)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# FEM con diferentes k
for idx, k_val in enumerate(k_conv_values):
    n_elem_range = [10, 20, 30, 40, 60, 80]
    errors_n = []
    
    for n_e in n_elem_range:
        nodes_t, h_t, conn_t = create_mesh(L, n_e)
        c_fem = solve_FEM(nodes_t, conn_t, D, U, k_val, c_inlet, quad_n=15)
        c_fem_interp = np.interp(x_exact, nodes_t, c_fem)
        c_ex = solucion_analitica(x_exact, D, U, k_val, L, c_inlet)
        error = np.sqrt(np.trapz((c_fem_interp - c_ex)**2, x_exact) / L)
        errors_n.append(error)
    
    axes[1].loglog(n_elem_range, errors_n, 's-', lw=2, label=f'k = {k_val:.2f} 1/hr')

axes[1].set_xlabel('Número de elementos', fontsize=12)
axes[1].set_ylabel('Error L2', fontsize=12)
axes[1].set_title('Convergencia FEM para diferentes k', fontsize=13)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Análisis y Respuestas a las Preguntas

### 9.1 ¿Qué efectos tiene computacionalmente aplicar una discretización?

**Análisis:**

1. **Estructura de matrices:**
   - **No discretizada (Legendre)**: Matriz densa $M \times M$
   - **Discretizada (FEM)**: Matriz banda (sparse) con ancho limitado

2. **Costo computacional:**
   - **No discretizada**: $O(M^3)$ para resolver, $O(M^2)$ para ensamblar
   - **Discretizada**: $O(N)$ para ensamblar (N = número de nodos), resolución más eficiente con sparse solvers

3. **Escalabilidad:**
   - **No discretizada**: Limitada por ortogonalidad numérica de funciones base globales
   - **Discretizada**: Excelente escalabilidad, puede manejar millones de elementos

4. **Adaptabilidad:**
   - **No discretizada**: Refinamiento uniforme (aumentar M)
   - **Discretizada**: Refinamiento local (mesh adaptivity)

### 9.2 ¿Con cuál de las dos estrategias se tiene un orden de convergencia mayor?

**Análisis:**

Del estudio de convergencia observamos:

- **Legendre (no discretizada)**: Convergencia **exponencial** (espectral) para soluciones suaves
  - Error decrece rápidamente con M
  - Mejor para problemas con soluciones suaves y dominios simples

- **Lagrange FEM (discretizada)**: Convergencia **algebraica** $O(h^p)$ donde $p$ = orden del elemento
  - Para elementos lineales: $O(h^2)$
  - Más robusto para problemas complejos

**Conclusión**: Legendre tiene **mayor orden de convergencia** para este problema suave, pero FEM es más práctico para problemas reales.

### 9.3 ¿Qué efectos tiene reducir y aumentar k?

**Efectos de reducir k (menor tasa de reacción):**
- Menor consumo de reactivo
- Mayor concentración a la salida
- Menor conversión
- Perfil de concentración más plano

**Efectos de aumentar k (mayor tasa de reacción):**
- Mayor consumo de reactivo
- Menor concentración a la salida
- Mayor conversión
- Perfil de concentración más pronunciado
- Gradientes más fuertes (puede requerir mayor refinamiento numérico)

**Implicaciones numéricas:**
- Para k grande: pueden aparecer capas límite que requieren refinamiento local
- Número de Péclet advectivo-reactivo: $Pe_r = \frac{UL}{\sqrt{kD}}$
- Para mantener precisión similar, se requiere mayor refinamiento con k grande

## 10. Resumen de Resultados y Conclusiones

In [ ]:
print("="*70)
print("RESUMEN DE RESULTADOS - REACTOR PFR")
print("="*70)

print("\n1. PARÁMETROS DEL PROBLEMA:")
print(f"   D = {D} m²/hr")
print(f"   U = {U} m/hr")
print(f"   k = {k} 1/hr")
print(f"   L = {L} m")
print(f"   c_inlet = {c_inlet} mol/L")

print("\n2. MÉTODO NO DISCRETIZADO (LEGENDRE):")
print(f"   - Convergencia: Exponencial (espectral)")
print(f"   - M óptimo: ~12-15 funciones base")
print(f"   - Ventaja: Alta precisión con pocos grados de libertad")
print(f"   - Desventaja: Matrices densas, limitado a geometrías simples")

print("\n3. MÉTODO DISCRETIZADO (LAGRANGE FEM):")
print(f"   - Convergencia: Algebraica O(h²) para elementos lineales")
print(f"   - Elementos recomendados: 40-60 para buena precisión")
print(f"   - Ventaja: Escalable, adaptable, matrices sparse")
print(f"   - Desventaja: Requiere más grados de libertad")

print("\n4. ESTUDIO PARAMÉTRICO DE k:")
for k_val in [k/2, k, 2*k]:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    conv = (c_inlet - c_out)/c_inlet * 100
    print(f"   k = {k_val:.2f} 1/hr: c(L) = {c_out:.2f} mol/L, Conversión = {conv:.1f}%")

print("\n5. CONCLUSIONES:")
print("   - Ambos métodos convergen correctamente a la solución analítica")
print("   - Legendre es más eficiente para este problema 1D suave")
print("   - FEM sería preferible para geometrías complejas o 2D/3D")
print("   - Mayor k requiere mayor refinamiento numérico")
print("="*70)